In [1]:
from IPython.display import clear_output
!pip install -e git+https://github.com/projectmesa/mesa#egg=mesa
import mesa
!pip install ipynb
import ipynb
clear_output(wait=True)
print("Everything A-Okay!")

Everything A-Okay!


In [6]:
import numpy as np
import math

In [3]:
from mesa import Model
from mesa.space import ContinuousSpace

class TestModel(Model):
    def __init__(self, width, height):
        self.height = width
        self.width = height
        
        # Create continuous space
        self.space = ContinuousSpace(self.width, self.height, torus=False, x_min=0, y_min=0)
        
        self.n_agents = 0
        self.agents = []

    
    def new_agent(self, agent_type, pos):
        '''
        Method that enables us to add agents of a given type.
        '''
        self.n_agents += 1
        
        # Create a new agent of the given type
        new_agent = agent_type(self.n_agents, self, pos)
        
        # Place the agent on the grid
        self.space.place_agent(new_agent, pos)
        
        # And add the agent to the model so we can track it
        self.agents.append(new_agent)
        
    def remove_agent(self, agent):
        '''
        Method that enables us to remove passed agents.
        '''
        self.n_agents -= 1
        
        # Remove agent from grid
        self.space.remove_agent(agent)
        
        # Remove agent from model
        self.agents.remove(agent)
        
    def step(self):
        '''
        Method that steps every agent. 
        
        Prevents applying step on new agents by creating a local list.
        '''
        for agent in list(self.agents):
            agent.step()

In [13]:
from mesa import Agent
import random

class Monkey(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)

        self.pos = pos
#         self.vis_range = 3
#         self.vision_angle = 3.14
        self.angle = 90

    def move(self, new_pos):
        '''
        This method should get the neighbouring cells (Moore's neighbourhood), select one, and move the agent to this cell.
        '''
        
        # Move agent to the new position
        self.model.grid.move_agent(self, new_pos)

    def pedestrians_in_field(self, vision_angle, vis_range):
        """
        returns the number of pedestrians in the field
        """
        # Calculate the lower angle and the upper angle
        lower_angle = self.angle - (vision_angle / 2)
        upper_angle = self.angle + (vision_angle / 2)

        # Change the current points to an np array for simplicity
        p0 = np.array(self.pos)

        # Convert to radians for angle calcuation
        u_rads = math.radians(upper_angle)
        l_rads = math.radians(lower_angle)

        # Calculate the end angles
        dx1 = math.cos(l_rads) * vis_range
        dy1 = math.sin(l_rads) * vis_range
        dx2 = math.cos(u_rads) * vis_range
        dy2 = math.sin(u_rads) * vis_range

        
        # Calculate the points
        p1 = np.array([p0[0] + dx1, p0[1] + dy1])
        p2 = np.array([p0[0] + dx2, p0[1] + dy2])
        
        # Calculate the vectors
        v1 = p1-p0
        v2 = p2-p1
        print('v1:', v1, 'v2:', v2)

        # Get the current neighbors
        neighbours = self.model.space.get_neighbors(self.pos, include_center=False, radius=vis_range)
        cone_neigh = []
        # Loop to find if neighbor is within the cone
        for neigh in neighbours:
            v3 = np.array(neigh.pos) - p0
            print('neigh.pos:', np.array(neigh.pos), 'v3:',v3)
            check1 = round(np.cross(v1, v3) * np.cross(v1, v2), 10)
            check2 = round(np.cross(v2, v3) * np.cross(v2, v1), 10)

            if (np.cross(v1, v3) * np.cross(v1, v2) >= 0 and np.cross(v2, v3) * np.cross(v2, v1) >= 0 and type(neigh) == Monkey):
                print("We are in the vis_range")
                print(self.pos)
                print(neigh.pos)
                cone_neigh.append(neigh)
            else:
                print("We are out of the vis_range")
                print(self.pos)
                print(neigh.pos)

        return cone_neigh


In [14]:
def test_field(pos_1, pos_2, vision_range):
    tester = TestModel(10, 10)

    # Create a Monkey
    tester.new_agent(Monkey, pos_1)
    tester.new_agent(Monkey, pos_2)

    # Create a reference, so that we can properly test
    Bernard = tester.agents[0]
    print("Bernard", Bernard.pos)

    Babette = tester.agents[1]
    print('Babette', Babette.pos)

    # # Move Bernard to a new position
    # new_position = (2.2, 1)
    # Bernard.move(new_position)
    # print(Bernard.pos)
    return Bernard.pedestrians_in_field(180, vision_range)


In [15]:
# VISION FIELD SET AT 180 DEGREES
# ASSUMING THE CORRECT MAP
# VISION RANGE = 1

# Check if for bernard at 0,0, he sees babette at 1/2*sqrt(2),1/2*sqrt(2), which corresponds to radius of 1/4pi
assert len(test_field((0,0), ((2)**.5/2-.001, (2)**.5/2-.001), 1))>0, 'Bernard cannot see Babette! 1 :(('
print()
# Check if for bernard at 5,5, he sees babette at 5+1/2*sqrt(2),5+1/2*sqrt(2), which corresponds to radius of 1/4pi
assert len(test_field((5,5), (5+(2)**.5/2-.001, 5+(2)**.5/2-.001), 1))>0, 'Bernard cannot see Babette! 2 :(('

print()
# Check if for bernard at 5,5, he sees babette at 5-1/2*sqrt(2),5+1/2*sqrt(2), which corresponds to radius of 3/4pi
assert len(test_field((5,5), (5-((2)**.5/2-.001), 5+((2)**.5/2-.001)), 1))>0, 'Bernard cannot see Babette! 3 :(('

Bernard (0, 0)
Babette (0.7061067811865476, 0.7061067811865476)
v1: [1. 0.] v2: [-2.0000000e+00  1.2246468e-16]
neigh.pos: [0.70610678 0.70610678] v3: [0.70610678 0.70610678]
We are in the vis_range
(0, 0)
(0.7061067811865476, 0.7061067811865476)

Bernard (5, 5)
Babette (5.706106781186548, 5.706106781186548)
v1: [1. 0.] v2: [-2.  0.]
neigh.pos: [5.70610678 5.70610678] v3: [0.70610678 0.70610678]
We are in the vis_range
(5, 5)
(5.706106781186548, 5.706106781186548)

Bernard (5, 5)
Babette (4.293893218813452, 5.706106781186548)
v1: [1. 0.] v2: [-2.  0.]
neigh.pos: [4.29389322 5.70610678] v3: [-0.70610678  0.70610678]
We are in the vis_range
(5, 5)
(4.293893218813452, 5.706106781186548)
